In [6]:
import csv
import pandas as pd

In [7]:
def extractDict(nameFile):
    dic = {}
    with open(nameFile) as file:
        reader = csv.reader(file)
        if len(reader.__next__()) > 2:
            for row in reader:
                key = row[0]
                if key in dic.keys():
                    dic[key] += [row[1:]]
                else:
                    dic[key] = [row[1:]]
        else:
            for row in reader:
                key = row[0]
                dic[key] = row[1:][0]
    return dic

In [8]:
def summarize(entity, dictElement):
    textPositive, textNegative, wordsPolarity = [], [], []
    totalTexts, countPositive, countNegative, dominance = 0, 0, 0, 0
    for element in dictElement[entity]:
        totalTexts += 1 
        sentiment = int(element[1]) + int(element[2])
        dominance += sentiment
        if sentiment > -1:
            countPositive = countPositive + 1
            textPositive += [element[0]]
        else:
            countNegative = countNegative + 1
            textNegative += [element[0]]
        for word in element[3].strip('[]').replace("'","").split(", "):
            if word not in wordsPolarity and len(word)>2:
                wordsPolarity.append(word)
    return [textPositive, countPositive, textNegative, countNegative, totalTexts, dominance, wordsPolarity]

In [9]:
dictEntities = extractDict('../listRelacion.csv')
dictSentence = extractDict('../identifySentiment/sentencesPolarity.csv')

In [13]:
#lstSummary = [['Entidad', 'Descripción', 'Predominancia',
#               'Enunciados postivos', 'Total de enunciados positivos',
#               'Enunciados negativos', 'Total de enunciados negativos',
#               'Palabras que dan la carga', 'Total de enunciados']]
lstEntity, lstDescription, lstDominance, lstTextPositive, lstCountPositive = [], [], [], [], []
lstTextNegative, lstCountNegative, lstWordsPolarity, lstTotalTexts = [], [], [], []
for (description, entity) in dictEntities.items():
    textPositive, textNegative, wordsPolarity = [], [], []
    countPositive, countNegative, totalTexts, dominance = 0, 0, 0, 0 
    if entity in dictSentence.keys():
        [textPositive, countPositive, textNegative,
         countNegative, totalTexts, dominance, wordsPolarity] = summarize(entity, dictSentence)
    if dominance > 0:
        dominance = 'Positiva' 
    elif dominance < 0:
        dominance = 'Negativa'
    else:
        dominance = 'Neutra'
        
    lstEntity.append(entity)
    lstDescription.append(description)
    lstDominance.append(dominance)
    lstTextPositive.append(textPositive)
    lstCountPositive.append(countPositive)
    lstTextNegative.append(textNegative)
    lstCountNegative.append(countNegative)
    lstWordsPolarity.append(wordsPolarity)
    lstTotalTexts.append(totalTexts)

df = pd.DataFrame({'Entidad': lstEntity, 'Descripción': lstDescription, 'Predominancia': lstDominance,
               'Enunciados postivos': lstTextPositive, 'Total de enunciados positivos': lstCountPositive,
               'Enunciados negativos': lstTextNegative, 'Total de enunciados negativos': lstCountNegative,
               'Palabras que dan la carga': lstWordsPolarity, 'Total de enunciados': lstTotalTexts})

In [19]:
writer = pd.ExcelWriter('summarySentence.xlsx', engine = 'xlsxwriter')
df.to_excel(writer, sheet_name = 'Sheet1', index = False)
writer.save()